In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import seaborn as sns

In [3]:
import os
import sys

In [4]:
# chaging directory

In [5]:
root, *_ = !pwd

In [6]:
if root.endswith('notebooks'):
    os.chdir('../')
    root, *_ = !pwd
    sys.path.append('src/')

In [7]:
from pipeline.core import *

In [8]:
os.getcwd()

'/home/med/projects/competitions/mlb-kaggle'

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
RAW_PATH = Path(os.environ['RAW_PATH'])

In [11]:
os.listdir(RAW_PATH)

['seasons.csv',
 'teams.csv',
 'example_sample_submission.csv',
 'train.csv',
 'mlb-player-digital-engagement-forecasting.zip',
 'awards.csv',
 'players.csv',
 'example_test.csv',
 'mlb']

In [12]:
train_data = pd.read_csv(RAW_PATH / 'train.csv', parse_dates=['date'])

In [13]:
train_data.head()

,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,2018-01-01,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,2018-01-02,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,2018-01-03,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,2018-01-04,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,2018-01-05,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [14]:
train_data = train_data.set_index('date')

### calculating mean 

In [15]:
# transforming

In [16]:
# this has to be replace for the .py code
from pipeline.core import *

In [17]:
target_name = 'nextDayPlayerEngagement'

In [18]:
unpack_target_tmf = make_unpack_tmf('nextDayPlayerEngagement')

In [19]:
unpack_target_tmf

PdColumnTransformer(transformers=[('FunctionTransformer',
                                   FunctionTransformer(func=<function unpack_column at 0x7f13d90fe670>),
                                   'nextDayPlayerEngagement')])

In [20]:
target = unpack_target_tmf.fit_transform(train_data)

In [21]:
target

,engagementMetricsDate,playerId,target1,target2,target3,target4,date
0,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294,2018-01-01
1,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118,2018-01-01
2,2018-01-02,519317,0.974327,56.177043,13.693746,64.166667,2018-01-01
3,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745,2018-01-01
4,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373,2018-01-01
...,...,...,...,...,...,...,...
2506171,2021-05-01,451661,0.000000,0.013314,0.000000,0.625925,2021-04-30
2506172,2021-05-01,519301,0.000131,0.003329,0.000000,0.216229,2021-04-30
2506173,2021-05-01,527055,0.000000,0.019971,0.000000,0.273131,2021-04-30
2506174,2021-05-01,543484,0.000131,0.056586,0.000000,1.024240,2021-04-30


In [22]:
from evaluate.metrics import *

In [23]:
from data.split import SplitData

In [24]:
sp = SplitData("2021-04-01", train_days=120)

In [25]:
train_df = sp.train(target)
valid_df = sp.valid(target)

In [26]:
train_df.date.agg(('min', 'max'))

min   2020-12-02
max   2021-03-31
Name: date, dtype: datetime64[ns]

In [27]:
valid_df.date.agg(('min', 'max'))

min   2021-04-01
max   2021-04-30
Name: date, dtype: datetime64[ns]

In [28]:
target_cols = [f'target{i}' for i in range(1, 5)];
index_cols = ['date', 'playerId'] 
target_cols, index_cols

(['target1', 'target2', 'target3', 'target4'], ['date', 'playerId'])

In [29]:
class MeanEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, ids: str = ['playerId'],
                 features: str = ['target1', 'target2',
                                  'target3', 'target4']):
        self.ids = ids
        self.features = features
    
    def fit(self, X: pd.DataFrame, y=None):
        self.mean = X.groupby(self.ids)[self.features].median().reset_index()
    
    def predict(self, X):
        
        drop_features = [feature for feature in self.features
                         if feature in X.columns]
        X = X.drop(drop_features, axis=1)
        output = X.merge(self.mean, on=self.ids)
        output = output[self.features]
        output.columns = output.columns.str.replace('target', 'yhat')
        return output

In [30]:
model= MeanEncoding()

In [31]:
model.fit(train_df)

In [32]:
valid_prediction = model.predict(valid_df)

In [33]:
for name, value in valid_prediction.items():
    valid_df[name] = value

In [34]:
compute_metrics(valid_df)

{'mae_mean': 1.449488128580136,
 'mae_1': 0.8568508711370514,
 'mae_2': 2.4792356236716744,
 'mae_3': 0.6010446554564337,
 'mae_4': 1.8608213640553846}

In [35]:
compute_metrics(valid_prediction[target_cols].values,
                valid_df[target_cols].values)

KeyError: "None of [Index(['target1', 'target2', 'target3', 'target4'], dtype='object')] are in the [columns]"